In [ ]:

# importing essential libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder #encoder
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder # encoding to overcome the trap of assigning values according to category number
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


df = pd.read_csv('../input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')    

We will first encode the required data in order to be able to use it in the initial findings we gather using the data. We will be using the Group By function for this purpose. We will group by the HSC board , SSC Board, the stream chosen and also the gender of the student. The GroupBy results will only be based on the categorical variables. 

In [ ]:
encoder = LabelEncoder()
encoder1 = LabelEncoder()
df['status'] = encoder.fit_transform(df['status'])
df.groupby('ssc_b').mean()

 We clearly observe that the salary difference between Central and Other Boards is not significant. However, Other boards are marginally better than Central Board in terms of Placement percentage and salary

In [ ]:
df.groupby('gender').mean()

This shows the difference  between the average salaries of males and females. There is a remarkable difference of aboyr 22,700 units of currency. Also a significant 8 percent of difference in placement probabilities . Females have a lower chance of getting a job and even if they do, they are paid less despite scoring more than men almost throughout the educational process.

In [ ]:
print(df.groupby('hsc_b').mean()) # grouped by HSC board
df.groupby('hsc_s').mean() # grouped by HSC Stream ie Science, comm and arts


We can see that Science and commerce are almost neck and neck in terms of jobs occupied and salary availed while Arts lags behind significatly

In [ ]:
df.groupby('degree_t').mean()

The differences in degrees and streams is shown above. Science and Tech is clearly the best industry

In [ ]:
df.groupby('specialisation').mean()

Marketing and Finance is clearly the more rewarding option. 
We will now encode the data and categorize the descriptive data

In [ ]:
df.iloc[:,1] = encoder1.fit_transform(df.iloc[:,1])
df.iloc[:,3] = encoder1.fit_transform(df.iloc[:,3])
df.iloc[:,5] = encoder1.fit_transform(df.iloc[:,5])
df.iloc[:,6] = encoder1.fit_transform(df.iloc[:,6])
df.iloc[:,8] = encoder1.fit_transform(df.iloc[:,8])
df.iloc[:,9] = encoder1.fit_transform(df.iloc[:,9])
df.iloc[:,11] = encoder1.fit_transform(df.iloc[:,11])

We donot require the serial number as it contributes to nothing. We will thus omit the serial number. Also, for now we are only interested in the classification of the candidate getting a job or not. So, we will also omit the regression column with exact salary data. and One Hot Encode the columns with more than two categories.

In [ ]:
df = df.iloc[:,1:]
df_chance = df.iloc[:,:13]
enc = OneHotEncoder()
enc1 = OneHotEncoder()

We will now reshape the data and complete the prerequisites . We will scale the values that are continuous to make them normal

In [ ]:
array1 = df_chance.iloc[:,5].values
array2 = df_chance.iloc[:,5].values
enc.fit(array1.reshape(-1,1))
enc1.fit(array1.reshape(-1,1))
enc1.categories_
enc.categories_
array1 = enc.transform(array1.reshape(-1,1)).toarray()
array2 = enc.transform(array2.reshape(-1,1)).toarray()
df_new = pd.DataFrame(array1)
df_chance = df_chance.join(df_new)
df_chance = df_chance.drop(columns = 0, axis = 1)
df_chance = df_chance.rename(columns={1: 'new_1', 2: "new_2"})
df_new = pd.DataFrame(array2)
df_chance = df_chance.join(df_new)
df_chance = df_chance.drop(columns = 0, axis = 1)
df_chance = df_chance.drop(columns = 'hsc_s', axis = 1)
df_chance = df_chance.drop(columns = 'degree_t', axis = 1)
df_chance = df_chance.rename(columns={1: 'new_3', 2: "new_4"})
#df_chance.iloc[:,5] = enc.fit_transform(df_chance.iloc[:,5]).toarray()
result = df['status'].values
df_chance = df_chance.drop(columns = 'status', axis = 1)

scaler = StandardScaler()
df_chance = df_chance.values
df_chance[:,[1,3,5,7,9]] = scaler.fit_transform(df_chance[:,[1,3,5,7,9]])

We split the train and test sets to carry out the classification process. Intuitively, the process should give a better result than just random guess because of all the EDA that we carried out. The data is certainly insightful

In [ ]:
clf = SVC(gamma = 'auto')

X_train = df_chance[:172, :]
X_test = df_chance[172:, :]
y_train = result[:172]
y_test = result[172:]

clf.fit(X_train, y_train)
clf1 = RandomForestClassifier(n_estimators = 200)
clf1.fit(X_train, y_train)
preds = clf.predict(X_test)
pred1 = clf1.predict(X_test)
cm = confusion_matrix(y_test, preds)
cm1 = confusion_matrix(y_test, pred1)
print(cm)
print(cm1)

We have used two classification methods here. Namely, Random forest classifier and the Support vector classifier. The working of these classifers is very different. While one deals with a network of decision trees, the other deals with a boundary range or support range across the (n-1) dimensional plane for n dimensional data. We will now use the data we initially discarded in order to make regression predictions. Only if a candidate is placed, will he earn money. Thus, we use a for loop and create an array of the candidates who were placed and apply regression model to predict their salaries

In [ ]:
array2 = df.iloc[:,13].values

results2 = []
factors = []

for i in range(len(result)):
    if(result[i] == 1):
        results2.append(array2[i])
        factors.append(df_chance[i])
        
factors = np.array(factors)
results2 = np.array(results2)
results2 = scaler.fit_transform(results2.reshape(-1,1))

X_train = factors[:113, :]
X_test = factors[113:, :]
y_train = results2[:113]
y_test = results2[113:]
reg = LinearRegression()
reg.fit(X_train, y_train)
pred_lin = reg.predict(X_test)

regressor = SVR()
regressor.fit(X_train, y_train)
pred = regressor.predict(X_test)
score = mean_squared_error(y_test, pred)
score_1 = mean_squared_error(y_test, pred_lin)

In [ ]:
print(score)
print(score_1)

These are the mse values of the results that we have obtained. 

To summarize, we started with EDA and speculated the parameters that can contribute to the results. We then used classification to understand and predict the possibility of a candidate getting a job. Out of the candidates that might get a job, we also predicted their salaries using a regression model. For regression, we used support vector regression and linear regression